# ESG + rendements

Ce notebook genere les rendements mensuels via Yahoo Finance et les aligne aux scores ESG.


In [1]:
import pandas as pd
from pathlib import Path
import yfinance as yf


In [3]:
# Localiser le projet
p = Path.cwd()
project_root = None
for _ in range(6):
    if (p / 'pyproject.toml').exists():
        project_root = p
        break
    if p.parent == p:
        break
    p = p.parent

if project_root is None:
    raise FileNotFoundError('pyproject.toml not found')

data_path = project_root / 'data/clean/notebook1/panel_complet.csv'
esg = pd.read_csv(data_path)
esg['Date'] = pd.to_datetime(esg['Date'])


In [4]:
# Mapping Bloomberg -> Yahoo (manuel pour tickers non-US)
ticker_map = {
    '000660 KS Equity': '000660.KS',
    '2383 TT Equity': '2383.TW',
    '6857 JP Equity': '6857.T',
    '9984 JP Equity': '9984.T',
    'ENR GR Equity': 'ENR.DE',
    'CLS CN Equity': 'CLS',
    'FN US Equity': 'FN',
    'TSEM IT Equity': 'TSEM',
}

def to_yahoo_ticker(bb_ticker: str) -> str:
    if bb_ticker in ticker_map:
        return ticker_map[bb_ticker]
    return bb_ticker.split()[0]

esg['Yahoo_Ticker'] = esg['Ticker'].apply(to_yahoo_ticker)


In [5]:
# Telecharger prix ajustés et calculer rendements mensuels
start = esg['Date'].min().strftime('%Y-%m-%d')
end = esg['Date'].max().strftime('%Y-%m-%d')
tickers = sorted(esg['Yahoo_Ticker'].unique())

prices = yf.download(
    tickers=tickers,
    start=start,
    end=end,
    interval='1d',
    auto_adjust=True,
    group_by='ticker',
    progress=False,
)

records = []
if isinstance(prices.columns, pd.MultiIndex):
    for t in tickers:
        if t in prices.columns.get_level_values(0):
            px = prices[t]['Close'].rename('Price').to_frame()
            px['Yahoo_Ticker'] = t
            records.append(px)
    prices_long = pd.concat(records).reset_index()
else:
    prices_long = prices.rename('Price').to_frame().reset_index()
    prices_long['Yahoo_Ticker'] = tickers[0]

prices_long['Date'] = pd.to_datetime(prices_long['Date'])
prices_monthly = (
    prices_long
    .set_index('Date')
    .groupby('Yahoo_Ticker')['Price']
    .resample('ME')
    .last()
    .reset_index()
)
prices_monthly['Return'] = (
    prices_monthly
    .groupby('Yahoo_Ticker')['Price']
    .pct_change()
)


In [6]:
# Aligner ESG sur fin de mois et fusionner
esg_aligned = esg.copy()
esg_aligned['Date'] = esg_aligned['Date'] + pd.offsets.MonthEnd(0)

panel = esg_aligned.merge(
    prices_monthly[['Date','Yahoo_Ticker','Return']],
    on=['Date','Yahoo_Ticker'],
    how='left'
)


In [7]:
# Sauvegarder
out_dir = project_root / 'data/clean/notebook4'
out_dir.mkdir(parents=True, exist_ok=True)
panel.to_csv(out_dir / 'panel_esg_returns.csv', index=False)
panel.head()


,Date,Ticker,Env_Score,Soc_Score,ESG_Score,Entreprise,Yahoo_Ticker,Return
0,2015-12-31,000660 KS Equity,2.94,2.84,3.47,SK Hynix,000660.KS,NaN
1,2016-01-31,000660 KS Equity,2.94,2.84,3.47,SK Hynix,000660.KS,NaN
2,2016-02-29,000660 KS Equity,2.94,2.84,3.47,SK Hynix,000660.KS,0.098901
3,2016-03-31,000660 KS Equity,2.94,2.84,3.47,SK Hynix,000660.KS,-0.061667
4,2016-04-30,000660 KS Equity,2.94,2.84,3.47,SK Hynix,000660.KS,0.000000
